In [ ]:
import xml.etree.ElementTree as ET
file1 = open('data/epidemicsimulation.xml')
tree1 = ET.parse(file1)
root1 = tree1.getroot()
PRE_SAMPLE_CONST1 = 1

In [ ]:
maxTime1 = 100

samples1 = root1.findall("./sample")
if PRE_SAMPLE_CONST1 > 1:
    samples1 = samples1[0::PRE_SAMPLE_CONST1]
samples1 = filter(lambda sample: sample.attrib["spVLtrans"] != 'NaN', samples1)
samples1 = filter(lambda sample: float(sample.attrib["time"]) <= maxTime1, samples1)
times1 = [float(sample.attrib["time"]) for sample in samples1]

running_heritsSPVL1 = [float(sample.attrib["running_slopeSPVL"]) for sample in samples1]
running_meanSPVLs1 = [float(sample.attrib["running_meanSPVL"]) for sample in samples1]
running_incidence1 = [float(sample.attrib["running_incidence"]) for sample in samples1]
running_medianSPVL1 = [float(sample.attrib["running_medianSPVL"]) for sample in samples1]
running_top_percentileSPVL1 = [float(sample.attrib["running_top_percentileSPVL"]) for sample in samples1]
running_bottom_percentileSPVL1 = [float(sample.attrib["running_bottom_percentileSPVL"]) for sample in samples1]

running_heritsNoMuts1 = [float(sample.attrib["running_slopeNoMuts"]) for sample in samples1]
running_meanNoMuts1 = [float(sample.attrib["running_meanNoMuts"]) for sample in samples1]
running_medianNoMuts1 = [float(sample.attrib["running_medianNoMuts"]) for sample in samples1]
running_top_percentileNoMuts1 = [float(sample.attrib["running_top_percentileNoMuts"]) for sample in samples1]
running_bottom_percentileNoMuts1 = [float(sample.attrib["running_bottom_percentileNoMuts"]) for sample in samples1]

prevalences1 = [float(sample.attrib["prevalence"]) for sample in samples1]
popsizes1 = [float(sample.attrib["popsize"]) for sample in samples1]
spVLrec1 = [float(sample.attrib["spVLrec"]) for sample in samples1]
spVLtrans1 = [float(sample.attrib["spVLtrans"]) for sample in samples1]
numberOfMutationsTrans1 = [float(sample.attrib["numberOfMutationsTrans"]) for sample in samples1]
numberOfMutationsRec1 = [float(sample.attrib["numberOfMutationsRec"]) for sample in samples1]
dTrans1 = [float(sample.attrib["dTrans"]) for sample in samples1]
dRec1 = [float(sample.attrib["dRec"]) for sample in samples1]

popsize1 = root1.attrib["POPSIZE"]

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 18})

SAMPLE_CONST = 1

def percentify(xs): return [x*100 for x in xs]
def lighten(colarray,alpha):
    return [1-alpha*(1-x) for x in colarray]

blueList = [0.1,0.1,0.8]
blackList = [0,0,0]

alph1 = 0.2
alph2 = 0.6

fig = plt.figure(figsize=[8,10])
ax1 = fig.add_subplot(411)
ax2 = fig.add_subplot(412,sharex=ax1)
ax3 = fig.add_subplot(413,sharex=ax1)
ax4 = fig.add_subplot(414,sharex=ax1)

rel_prevalences1 = [pr/float(popsize1) for pr in prevalences1]
ax1.plot(times1,percentify(rel_prevalences1),color='black',linewidth=2.0)
ax1.set_ylabel('prevalence (%)')
ax1.set_ylim(percentify([0,1]))

ax2.plot(times1,running_meanSPVLs1,linewidth=2.0,color=blueList)
ax2.plot(times1,running_top_percentileSPVL1,linewidth=0.5,color=lighten(blueList,alph1))
ax2.plot(times1,running_bottom_percentileSPVL1,linewidth=0.5,color=lighten(blueList,alph1))
ax2.fill(times1 + times1[::-1],running_top_percentileSPVL1 + running_bottom_percentileSPVL1[::-1],color=lighten(blueList,alph1))
ax2.scatter([times1[i] for i in range(0,len(times1),SAMPLE_CONST)],[spVLtrans1[i] for i in range(0,len(times1),SAMPLE_CONST)],0.5,color=lighten(blueList,alph2),zorder=2)
ax2.set_ylim([2,10])
ax2.set_ylabel("spVL")
##### draw a line at 4.52 in the spVL graph
ax2.plot(times1,[4.52 for _ in times1],linewidth=2.0,color='black',linestyle='--')

ax3.plot(times1,running_meanNoMuts1,linewidth=2.0,color=blackList)
ax3.plot(times1,running_top_percentileNoMuts1,linewidth=0.5,color=lighten(blackList,alph1))
ax3.plot(times1,running_bottom_percentileNoMuts1,linewidth=0.5,color=lighten(blackList,alph1))
ax3.fill(times1 + times1[::-1],running_top_percentileNoMuts1 + running_bottom_percentileNoMuts1[::-1],color=lighten(blackList,alph1))
ax3.scatter([times1[i] for i in range(0,len(times1),SAMPLE_CONST)],[numberOfMutationsTrans1[i] for i in range(0,len(times1),SAMPLE_CONST)],0.2,color=lighten(blackList,alph2),zorder=2)
ax3.set_ylim([0,60])
ax3.set_ylabel("# mutations")

ax4.plot(times1,percentify(running_heritsSPVL1),color="black",linewidth=0.5)
ax4.set_ylim(percentify([-0.1,0.4]))
ax4.set_ylabel("heritability (%)")
ax4.set_xlabel('time (years)')
ax4.set_xlim([0,maxTime1])
ax4.set_xticks(range(0,int(maxTime1)+1,int(maxTime1+1)/10))

### align the y-labels
labelxpos = -0.08
ax1.yaxis.set_label_coords(labelxpos, 0.5)
ax2.yaxis.set_label_coords(labelxpos, 0.5)
ax3.yaxis.set_label_coords(labelxpos, 0.5)
ax4.yaxis.set_label_coords(labelxpos, 0.5)

plt.setp(ax1.get_xticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.setp(ax3.get_xticklabels(), visible=False)

fig.tight_layout()

fig.savefig('epidemic-plot.png')